In [1]:
import pandas as pd
import numpy as np
from epiweeks import Week
import dask.dataframe as dd
from datetime import datetime

In [2]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_dep = {
    "AMAZONAS"	:1,
    "ANCASH"	:2,
    "APURIMAC"	:3,   
    "AREQUIPA"	:4,
    "AYACUCHO"	:5,   
    "CAJAMARCA"	:6,
    "CALLAO"	:7,
    "CUSCO"	    :8,
    "HUANCAVELICA":9,	
    "HUANUCO"	:10,	
    "ICA"	    :11,	
    "JUNIN"	    :12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA"	    :15,
    "LORETO"	:16,
    "MADRE DE DIOS":17,
    "MOQUEGUA"	:18,	
    "PASCO"	    :19,
    "PIURA"	    :20,
    "PUNO"	    :21,
    "SAN MARTIN":22,
    "TACNA"	    :23,
    "TUMBES"	:24,
    "UCAYALI"	:25
}

In [3]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['id_persona','FECHA_FALLECIMIENTO', 'EDAD_DECLARADA', 'SEXO', 'DEPARTAMENTO']
df_fal = pd.read_csv(url_fal, sep = ';', usecols = col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32', 
                                                                     'EDAD_DECLARADA': 'int8'})

In [4]:
# Ordenar columnas
df_fal.rename(columns = {'FECHA_FALLECIMIENTO':'fecha_fallecimiento', 'EDAD_DECLARADA': 'edad',
                         'SEXO': 'sexo', 'DEPARTAMENTO': 'departamento'}, inplace=True)

df_fal.drop(df_fal[df_fal.sexo == '.'].index, inplace = True)
df_fal.drop(df_fal[df_fal.edad == -9].index, inplace = True)  # un -9 en edad

# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')

def days_between(d1, d2):
    return abs((d2 - d1).days)


df_fal.loc[:,'fecha_fallecimiento'] = pd.to_datetime(df_fal['fecha_fallecimiento'], format = '%Y%m%d')  # Transformar a formato fecha
df_fal['edad_cat'] = edad_cat(df_fal, 'edad')  # Convertir edad en categoría
df_fal['sexo'] = df_fal['sexo'].replace(dic_sex).astype('int8')  # Reemplazar sexo con un code
df_fal['departamento'] = df_fal['departamento'].map(dic_dep).astype('int8')  # Codificar variables

fecha_min = df_fal['fecha_fallecimiento'].min()  # Agregar la diferencia en días desde el primer caso
df_fal['time_days'] = df_fal['fecha_fallecimiento'].apply(lambda row : days_between(row, fecha_min)+1)  # Agregar el num de días
# Agregar la semana epidemiológica y el año
df_fal[['year','epi_week']] = df_fal['fecha_fallecimiento'].apply(lambda date : Week.fromdate(date) \
                                                                                    .weektuple()) \
                                                                                    .tolist()
# Reorder
df_fal = df_fal[['id_persona', 'fecha_fallecimiento','year', 'epi_week', 'time_days', 'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['fecha_fallecimiento']) \
         .reset_index(drop = True)

In [5]:
df = dd.from_pandas(df_fal, npartitions=3)
del df_fal
df.head(10)

,id_persona,fecha_fallecimiento,year,epi_week,time_days,sexo,edad,edad_cat,departamento
0,36527082.0,2020-03-03,2020,10,1,1,63,5,22
1,36401995.0,2020-03-04,2020,10,2,1,85,7,22
2,36853977.0,2020-03-07,2020,10,5,2,24,1,16
3,NaN,2020-03-11,2020,11,9,2,78,6,20
4,NaN,2020-03-14,2020,11,12,1,8,0,16
5,36614476.0,2020-03-14,2020,11,12,1,4,0,15
6,36731113.0,2020-03-16,2020,12,14,1,57,4,15
7,36645051.0,2020-03-16,2020,12,14,2,88,7,15
8,36437890.0,2020-03-16,2020,12,14,1,73,6,15
9,24907449.0,2020-03-16,2020,12,14,2,26,1,5


In [6]:
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['id_persona','fecha_vacunacion','dosis', 'id_vacuna']
ddf_vac = dd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 'dosis': 'int8',
                                                                      'id_vacuna': 'int8', 'id_persona': 'float64'})

ddf_vac = ddf_vac.rename(columns = {'fecha_vacunacion': 'fecha_vacuna'})
ddf_vac = ddf_vac[['id_persona', 'dosis', 'fecha_vacuna', 'id_vacuna']]

ddf_vac.head()

,id_persona,dosis,fecha_vacuna,id_vacuna
0,505864.0,2,20211209,5
1,7676322.0,2,20211111,6
2,7752439.0,1,20211216,5
3,7638125.0,1,20210913,6
4,7507035.0,2,20211008,6


In [7]:
df_dosis_1 = ddf_vac[ddf_vac.dosis == 1]
df_dosis_1 = df_dosis_1.rename(columns = {'dosis' : 'dosis_1', 
                                          'fecha_vacuna': 'fecha_vacuna_1', 
                                          'id_vacuna': 'id_vacuna_1'})
df = df.merge(df_dosis_1, on = 'id_persona', how = 'left')
del df_dosis_1

df_dosis_2 = ddf_vac[ddf_vac.dosis == 2]
df_dosis_2['dosis'] = df_dosis_2['dosis'].replace(2, 1)
df_dosis_2 = df_dosis_2.rename(columns = {'dosis' : 'dosis_2', 
                                          'fecha_vacuna': 'fecha_vacuna_2', 
                                          'id_vacuna': 'id_vacuna_2'})
df = df.merge(df_dosis_2, on = 'id_persona', how = 'left')
del df_dosis_2

df_dosis_3 = ddf_vac[ddf_vac.dosis == 3]
df_dosis_3['dosis'] = df_dosis_3['dosis'].replace(3, 1)
df_dosis_3 = df_dosis_3.rename(columns = {'dosis' : 'dosis_3', 
                                          'fecha_vacuna': 'fecha_vacuna_3', 
                                          'id_vacuna': 'id_vacuna_3'})
df = df.merge(df_dosis_3, on = 'id_persona', how = 'left')
del df_dosis_3, ddf_vac

In [8]:
df = df.compute()

In [9]:
# Variable "vacunado" si recibe las 2 dosis
df['vacunado'] = np.where(df['dosis_2'] == 1, 1, 0).astype('int8')

df['dosis_1'] = df['dosis_1'].fillna(0).astype('int8')
df['dosis_2'] = df['dosis_2'].fillna(0).astype('int8')
df['dosis_3'] = df['dosis_3'].fillna(0).astype('int8')

df['id_vacuna_1'] = df['id_vacuna_1'].fillna(0).astype('int8')
df['id_vacuna_2'] = df['id_vacuna_2'].fillna(0).astype('int8')
df['id_vacuna_3'] = df['id_vacuna_3'].fillna(0).astype('int8')

In [10]:
df = df[['id_persona', 'fecha_fallecimiento', 'year', 'epi_week', 'time_days',
       'sexo', 'edad', 'edad_cat', 'departamento', 'vacunado', 'dosis_1', 'fecha_vacuna_1',
       'id_vacuna_1', 'dosis_2', 'fecha_vacuna_2', 'id_vacuna_2', 'dosis_3',
       'fecha_vacuna_3', 'id_vacuna_3']]

In [11]:
df = df.sort_values(by = 'fecha_fallecimiento').reset_index(drop = True)

In [12]:
df

,id_persona,fecha_fallecimiento,year,epi_week,time_days,sexo,edad,edad_cat,departamento,vacunado,dosis_1,fecha_vacuna_1,id_vacuna_1,dosis_2,fecha_vacuna_2,id_vacuna_2,dosis_3,fecha_vacuna_3,id_vacuna_3
0,36527082.0,2020-03-03,2020,10,1,1,63,5,22,0,0,NaN,0,0,NaN,0,0,NaN,0
1,36401995.0,2020-03-04,2020,10,2,1,85,7,22,0,0,NaN,0,0,NaN,0,0,NaN,0
2,36853977.0,2020-03-07,2020,10,5,2,24,1,16,0,0,NaN,0,0,NaN,0,0,NaN,0
3,NaN,2020-03-11,2020,11,9,2,78,6,20,0,0,NaN,0,0,NaN,0,0,NaN,0
4,36614476.0,2020-03-14,2020,11,12,1,4,0,15,0,0,NaN,0,0,NaN,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203371,9346434.0,2022-01-14,2022,2,683,2,59,4,15,0,0,NaN,0,0,NaN,0,0,NaN,0
203372,11184318.0,2022-01-14,2022,2,683,1,64,5,15,1,1,20210225.0,6,1,20210318.0,6,0,NaN,0
203373,8196728.0,2022-01-14,2022,2,683,1,64,5,15,1,1,20210601.0,5,1,20210623.0,5,1,20211213.0,5
203374,17398883.0,2022-01-14,2022,2,683,1,64,5,8,1,1,20210602.0,5,1,20210625.0,5,0,NaN,0


In [13]:
df.to_csv('../Data/DP6_covid19-peru_info-fallecidos.csv', index = False)